In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import hvplot.pandas
from scipy.stats import linregress
import json


In [17]:
health_wrob_df=pd.read_csv("../resources/VAERSDATA1 (3).csv")
health_wrob_df.head()
max_age=health_wrob_df["AGE_YRS"].max()
print(max_age)

109


In [18]:
health_wrob_state = health_wrob_df.sort_values(by=["STATE","AGE_YRS"], ascending=[True,True])
health_wrob_state=health_wrob_state[health_wrob_state["AGE_YRS"]>=10]
health_wrob_state.head()

,VAERS_ID,RECVDATE,STATE,AGE_YRS,SEX,DIED,DATEDIED,RECOVD,VAX_DATE,ONSET_DATE,NUMDAYS,V_ADMINBY,VAX_MANU,HISTORY,ALLERGIES
17459,1906451,11/29/2021,AK,10,F,NaN,NaN,U,11/10/2021,11/10/2021,0.0,PVT,PFIZER\BIONTECH,NaN,"Pork, Peanuts"
17460,1906459,11/29/2021,AK,10,F,NaN,NaN,U,11/10/2021,11/10/2021,0.0,PVT,PFIZER\BIONTECH,NaN,NaN
17461,1906495,11/29/2021,AK,10,M,NaN,NaN,U,11/10/2021,11/10/2021,0.0,PVT,PFIZER\BIONTECH,NaN,NaN
17767,1944375,12/13/2021,AK,10,F,NaN,NaN,NaN,11/24/2021,11/24/2021,0.0,PUB,PFIZER\BIONTECH,NaN,NaN
17768,1944576,12/13/2021,AK,10,M,NaN,NaN,NaN,11/26/2021,11/26/2021,0.0,PUB,PFIZER\BIONTECH,NaN,NaN


In [44]:
# creating bins
bins = list(range(10, 111, 10))
labels=[f"{i}-{i+9}" for i in range(10,110,10)]
health_wrob_state["Age_Group"]=pd.cut(health_wrob_df["AGE_YRS"],bins=bins,labels=labels,right=False)
health_wrob_state.head()

,VAERS_ID,RECVDATE,STATE,AGE_YRS,SEX,DIED,DATEDIED,RECOVD,VAX_DATE,ONSET_DATE,NUMDAYS,V_ADMINBY,VAX_MANU,HISTORY,ALLERGIES,Age Group,Age_Group
17459,1906451,11/29/2021,AK,10,F,NaN,NaN,U,11/10/2021,11/10/2021,0.0,PVT,PFIZER\BIONTECH,NaN,"Pork, Peanuts",10-19,10-19
17460,1906459,11/29/2021,AK,10,F,NaN,NaN,U,11/10/2021,11/10/2021,0.0,PVT,PFIZER\BIONTECH,NaN,NaN,10-19,10-19
17461,1906495,11/29/2021,AK,10,M,NaN,NaN,U,11/10/2021,11/10/2021,0.0,PVT,PFIZER\BIONTECH,NaN,NaN,10-19,10-19
17767,1944375,12/13/2021,AK,10,F,NaN,NaN,NaN,11/24/2021,11/24/2021,0.0,PUB,PFIZER\BIONTECH,NaN,NaN,10-19,10-19
17768,1944576,12/13/2021,AK,10,M,NaN,NaN,NaN,11/26/2021,11/26/2021,0.0,PUB,PFIZER\BIONTECH,NaN,NaN,10-19,10-19


In [45]:
print(health_wrob_state.columns.tolist())

['VAERS_ID', 'RECVDATE', 'STATE', 'AGE_YRS', 'SEX', 'DIED', 'DATEDIED', 'RECOVD', 'VAX_DATE', 'ONSET_DATE', 'NUMDAYS', 'V_ADMINBY', 'VAX_MANU', 'HISTORY', 'ALLERGIES', 'Age Group', 'Age_Group']


In [47]:
health_wrob_age=health_wrob_state.sort_values(by=["Age_Group"],ascending=True)
health_wrob_age.drop(columns=["Age Group"], inplace=True)
health_wrob_age.head()

,VAERS_ID,RECVDATE,STATE,AGE_YRS,SEX,DIED,DATEDIED,RECOVD,VAX_DATE,ONSET_DATE,NUMDAYS,V_ADMINBY,VAX_MANU,HISTORY,ALLERGIES,Age_Group
17459,1906451,11/29/2021,AK,10,F,NaN,NaN,U,11/10/2021,11/10/2021,0.0,PVT,PFIZER\BIONTECH,NaN,"Pork, Peanuts",10-19
36978,2036788,1/14/2022,FL,14,M,NaN,NaN,N,1/9/2022,1/13/2022,4.0,PVT,PFIZER\BIONTECH,ADHD,NaN,10-19
36892,2019415,1/8/2022,FL,14,F,NaN,NaN,N,1/6/2022,1/8/2022,2.0,PVT,PFIZER\BIONTECH,none,nkda,10-19
36890,2019372,1/8/2022,FL,14,M,NaN,NaN,NaN,1/7/2022,1/8/2022,1.0,PHM,PFIZER\BIONTECH,NaN,NaN,10-19
36889,2019143,1/8/2022,FL,14,M,NaN,NaN,N,6/11/2021,6/12/2021,1.0,UNK,PFIZER\BIONTECH,Comments: List of non-encoded Patient Relevant...,NaN,10-19


In [60]:
state_counts = health_wrob_df["STATE"].value_counts()
total_count= health_wrob_df["VAERS_ID"].count()
print(state_counts)
print(total_count)

STATE
CA    87943
TX    47897
NY    45635
FL    45215
MI    35306
      ...  
PW        2
QW        2
XV        2
XL        1
Ca        1
Name: count, Length: 65, dtype: int64
896529
